In [ ]:
import pandas as pd
import numpy as np
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import random
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from numpy.random import seed
from tensorflow import set_random_seed
from sklearn.cross_validation import train_test_split
import gpflow
from sklearn.svm import SVR
from matplotlib import pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

In [ ]:
generalDat = pd.read_csv('hmm/finalMoodData.csv')
group1data = pd.read_csv('hmm/group1data.csv')
group2data = pd.read_csv('hmm/group2data.csv')
group3data = pd.read_csv('hmm/group3data.csv')
dataprofile = pd.read_csv('hmm/GMProfileDt.csv')
sias =  pd.read_csv('hmm/SIAS.csv')
groupings = pd.read_csv('hmm/GMeanMoodDt.csv')

In [ ]:
def one_hot_encoding(col):
    """
    col: is a 1D array or list
    purpose: Takes in a column of categorical data and returns an R*N array of a one hot encoding
    with R categories
    """
    values = array(col)
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    #print integer_encoded
    # binary encode
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    return onehot_encoded


In [ ]:
dataset = groupings
dataset = dataset.sort_values(['date_long'], ascending = True)
y = dataset.pop('pid')
X = dataset
#Stratified sampling
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=42)
#result1 = X_train.sort_values(['date_long'], ascending= True)
X_train = X_train[predictors]
print X_train.shape

#result2 = X_test.sort_values(['date_long'], ascending= True)
X_test = X_test[predictors]
dat = X_train.append(X_test)
dataset = dat[predictors]

#print dataset.shape
np.random.seed(0)

# load dataset
#dataset = read_csv('pollution.csv', header=0, index_col=0)
values = dataset.values
# integer encode direction

encoded = one_hot_encoding(values[:,1])
values = np.delete(values, 1, 1)
values = np.concatenate((values, encoded), axis=1)



encoded = one_hot_encoding(values[:,1].astype(str))
values = np.delete(values, 1, 1)
values = np.concatenate((values, encoded), axis=1)



# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
#print scaled.shape
# frame as supervised learning
#reframed = series_to_supervised(scaled, 1, 1)

# drop columns we don't want to predict
reframed = scaled
#print reframed
#print(reframed.shape)


# split into train and test sets
values = reframed
n_train_hours = X_train.shape[0]
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
# split into input and outputs
train_X, train_y = train[:, 1:], train[:, 0]
test_X, test_y = test[:, 1:], test[:, 0]


train_y = train_y.reshape((train_y.shape[0], 1))
test_y = test_y.reshape((test_y.shape[0], 1))
print train_X.shape, train_y.shape, test_X.shape, test_y.shape

### Gaussian Process Regression with RBF kernel

In [ ]:

dim = train_X.shape[1]
k = gpflow.kernels.RBF(dim, lengthscales= 20)
m = gpflow.gpr.GPR(X = train_X, Y = train_y ,kern = k )
m.optimize(maxiter=30)
predic = m.predict_f(test_X)
# make a prediction
pred = predic[0]

# invert scaling for forecast
inv_yhat = concatenate((pred, test_X), axis=1)
#print inv_yhat.shape
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print 'Test RMSE: %.3f' % rmse

### The SVM with RBF kernel

In [ ]:
clf = SVR(C=1.0, epsilon=0.1)
clf.fit(train_X, train_y) 
# make a prediction
predic = clf.predict(test_X)
pred = predic.reshape((test_X.shape[0], 1))

# invert scaling for forecast
inv_yhat = concatenate((pred, test_X), axis=1)
#print inv_yhat.shape
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print 'Test RMSE: %.3f' % rmse

### The Lasso

In [ ]:
clf = linear_model.Lasso(alpha=0.1)
clf.fit(train_X, train_y)
# make a prediction
predic = clf.predict(test_X)
pred = predic.reshape((test_X.shape[0], 1))

# invert scaling for forecast
inv_yhat = concatenate((pred, test_X), axis=1)
#print inv_yhat.shape
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print 'Test RMSE: %.3f' % rmse

### The Random Forest

In [ ]:
clf = RandomForestRegressor(max_depth=3, random_state=0)
clf.fit(train_X, train_y)
# make a prediction
predic = clf.predict(test_X)
pred = predic.reshape((test_X.shape[0], 1))

# invert scaling for forecast
inv_yhat = concatenate((pred, test_X), axis=1)
#print inv_yhat.shape
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print 'Test RMSE: %.3f' % rmse

### The following are models based on subgroups of participant cohort 

<h6> Group-based with Gaussian Processes</h6>

In [ ]:
table = []
for j in grp:
    for i in groupings[j].unique():
        data = groupings[groupings[j] == i]
        data = data.sort_values(['date_long'], ascending = True)
        y = data.pop('pid')
        X = data
        X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=42)
        #result1 = X_train.sort_values(['date_long'], ascending= True)
        X_train = X_train[predictors]
        #print X_train.shape

        #result2 = X_test.sort_values(['date_long'], ascending= True)
        X_test = X_test[predictors]
        #print X_test.head()
        dat = X_train.append(X_test)
        data = dat[predictors]
        #print data.shape
        np.random.seed(0)
        # load dataset
        #dataset = read_csv('pollution.csv', header=0, index_col=0)
        values = data.values
        # integer encode direction
        #encoder = LabelEncoder()
        #values[:,4] = encoder.fit_transform(values[:,4])
        encoded = one_hot_encoding(values[:,1])
        values = np.delete(values, 1, 1)
        values = np.concatenate((values, encoded), axis=1)

        encoded = one_hot_encoding(values[:,1].astype(str))
        values = np.delete(values, 1, 1)
        values = np.concatenate((values, encoded), axis=1)


        # ensure all data is float
        values = values.astype('float32')
        # normalize features
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled = scaler.fit_transform(values)
        #print scaled.shape

        # drop columns we don't want to predict
        reframed = scaled


        # split into train and test sets
        values = reframed
        n_train_hours = X_train.shape[0]
        train = values[:n_train_hours, :]
        test = values[n_train_hours:, :]
        # split into input and outputs
        train_X, train_y = train[:, 1:], train[:, 0]
        test_X, test_y = test[:, 1:], test[:, 0]
        # reshape input to be 3D [samples, timesteps, features]
        train_y = train_y.reshape((train_y.shape[0], 1))
        test_y = test_y.reshape((test_y.shape[0], 1))
        print train_X.shape, train_y.shape, test_X.shape, test_y.shape

        dim = train_X.shape[1]
        k = gpflow.kernels.RBF(dim, lengthscales= 20)
        m = gpflow.gpr.GPR(X = train_X, Y = train_y ,kern = k ) 
        m.optimize(maxiter=30)
        predic = m.predict_f(test_X)
        pred = predic[0]
        # make a prediction
        #yhat = model.predict(test_X)
        #test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
        # invert scaling for forecast
        inv_yhat = concatenate((pred, test_X), axis=1)
        #print inv_yhat.shape
        inv_yhat = scaler.inverse_transform(inv_yhat)
        inv_yhat = inv_yhat[:,0]
        # invert scaling for actual
        test_y = test_y.reshape((len(test_y), 1))
        inv_y = concatenate((test_y, test_X), axis=1)
        inv_y = scaler.inverse_transform(inv_y)
        inv_y = inv_y[:,0]
        # calculate RMSE
        rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
        print 'Test RMSE: %.3f' % rmse
        table.append({'class': j ,'group': i, 'RMSE':  rmse,  '#pid': len(y.unique()), 'dim':  data.shape, 'size': data.shape[0], 
                     'Y': inv_y, 'Pred':inv_yhat })

results = pd.DataFrame(table) 
print results

In [ ]:
#Getting summary results from subgroup performance

tal = []
for i in jung['class'].unique():
    h = jung[jung['class'] == i]
    h['wRMSE'] = (h['RMSE'] * h['size'])/ np.sum(h['size']) 
    #print h.head(n =10)
    tal.append({'class': i ,'WRMSE': np.sum(h['wRMSE']), 'SD': np.std(h['RMSE']) })


tally = pd.DataFrame(tal) 
print tally